In [1]:
import tiktoken
print(tiktoken.__version__)

0.8.0


In [2]:
tokenizer = tiktoken.get_encoding("gpt2")

In [3]:
with open("ch02/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_data = f.read()
    
enc_text = tokenizer.encode(raw_data)
print(len(enc_text))

5145


In [5]:
enc_sample = enc_text[50:]

In [7]:
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [9]:
for i in range(1, context_size + 1):
    context = enc_sample[:i]
    target  = enc_sample[i]
    
    print(context, "--->", target)

[290] ---> 4920
[290, 4920] ---> 2241
[290, 4920, 2241] ---> 287
[290, 4920, 2241, 287] ---> 257


In [10]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    target  = enc_sample[i]
    print(tokenizer.decode(context), "--->", tokenizer.decode([target]))

 and --->  established
 and established --->  himself
 and established himself --->  in
 and established himself in --->  a


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader

In [3]:
class GPTDatasetV2(Dataset):
    def __init__(self, text, tokenizer, max_length, stride):
        self.input_ids  = []
        self.target_ids = []

        token_ids = tokenizer.encode(text)
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk  = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]

            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]


In [4]:
def create_dataloader_v1(text, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")

    dataset = GPTDatasetV2(text, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset, 
        batch_size=batch_size, 
        shuffle=shuffle, 
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [7]:
with open("the-verdict.txt", "r", encoding="utf-8") as fin:
    raw_data = fin.read()

dataloader = create_dataloader_v1(raw_data, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)

first_batch = next(data_iter)

print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [8]:
dataloader = create_dataloader_v1(raw_data, batch_size=8, max_length=4, stride=4, shuffle=False)
data_iter = iter(dataloader)

first_batch = next(data_iter)

print(first_batch)

[tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]]), tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])]


In [10]:
import torch.nn as nn

vocab_size = 50257
embed_dim = 256

vocab = nn.Embedding(vocab_size, embed_dim)

In [11]:
data_loader = create_dataloader_v1(raw_data, batch_size=8, max_length=4, stride=4, shuffle=False)
data_iter = iter(dataloader)

first_batch = next(data_iter)

print(first_batch)

[tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]]), tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])]


In [12]:
inputs, target = first_batch
print("input shape", inputs.shape)

input shape torch.Size([8, 4])


In [13]:
token_embedding = vocab(inputs)
print(token_embedding.shape)

torch.Size([8, 4, 256])


In [14]:
context_length = 4
position_embedding_layer = nn.Embedding(context_length, embed_dim)
position_embeddings = position_embedding_layer(torch.arange(context_length))

In [15]:
print(position_embeddings.shape)

torch.Size([4, 256])


In [16]:
input_embeddings = token_embedding + position_embeddings

In [17]:
print(input_embeddings.shape)

torch.Size([8, 4, 256])
